In [26]:
# import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta


In [27]:
df = pd.read_csv('BA.csv')

df['Date'] = pd.to_datetime(df['Date'])

df.index = df['Date']

df.head()
df.describe()


,Open,High,Low,Close,Adj Close,Volume
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03
mean,243.365596,245.790724,240.910509,243.459889,232.934569,4.336390e+06
std,100.400782,101.663496,99.031425,100.380078,103.286000,2.916014e+06
min,105.120003,109.839996,102.099998,108.440002,97.979713,7.889000e+05
25%,144.139999,145.259998,142.994995,144.422497,128.582894,2.837675e+06
50%,235.375000,237.099999,234.314995,235.235000,224.305252,3.627750e+06
75%,346.279991,350.385010,341.934998,346.740006,338.587814,4.858500e+06
max,446.010010,446.010010,440.190002,440.619995,432.860138,3.692260e+07


In [28]:
df = df.drop(["Date"], axis=1)

In [29]:
#5.1. Step 1: extract technical indicators

#Define Technical Indicators
#_______________________________________________________________________

#Simple Moving Average  
sma = ta.SMA(df.Close,25)
df['MA25'] = sma
df = df.fillna(0)

# RSI
df["RSI"] = ta.RSI(df.Close,14)

# Stochastic K
df["Stoch_K"], df["Stoch_D"] = ta.STOCH(df["High"], df["Low"], df["Close"], fastk_period=14, slowk_period=3, slowk_matype=0, 
                                       slowd_period=3, slowd_matype=0)

# MACD
df['MACD'], df['MACDSignal'], df['MACDHist'] = ta.MACD(df.Close, fastperiod= 12, slowperiod= 26, signalperiod= 9)

# Williams R%
df["WILLR"] = ta.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14) 

#df["WILLR"] = (WILLR - WILLR.min())/(WILLR.max() - WILLR.min())
df.tail()

,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR
Date,,,,,,,,,,,,,,
2020-01-27,316.209991,321.260010,314.579987,316.600006,316.600006,7468100,328.293201,39.800736,39.610963,32.293929,-6.286475,-5.699115,-0.587360,-66.530010
2020-01-28,318.049988,319.359985,314.000000,316.559998,316.559998,5502200,327.615601,39.775639,39.323804,37.478964,-6.322034,-5.823699,-0.498335,-64.521934
2020-01-29,324.350006,327.179993,319.140015,322.019989,322.019989,12454700,327.376400,44.883662,39.474169,39.469645,-5.842293,-5.827418,-0.014875,-50.525550
2020-01-30,320.500000,325.350006,317.640015,323.299988,323.299988,6022000,326.806400,46.039141,47.928693,42.242222,-5.297740,-5.721482,0.423742,-41.166436
2020-01-31,321.750000,321.929993,316.989990,318.269989,318.269989,4996800,326.217200,42.287379,51.987834,46.463565,-5.211977,-5.619581,0.407604,-52.344512


In [30]:
# 5.2. Step 2: trend analysis using technical indicators

#make MA15 into a series
MA25 = df.iloc[:,6]

# Up trend is 1, No trend is 0, and Down trend is -1
df['Trend'] = MA25.rolling('5d').apply(lambda x: np.sign(x[-1] - x[0]))

df.tail()
#If close > MA25 and MA25 is rising for last 5 days then 1 (Uptrend)
#If close < MA25 and MA25 is declining for last 5 days then -1 (Downtrend)
#If neither are true then 0 (Notrend)
def f(row):
    if row['Close'] > row['MA25'] and row['Trend'] == 1:
        val = 1
    elif row['Close'] < row['MA25'] and row['Trend'] == -1:
        val = -1
    else:
        val = 0
    return val
df['Trend'] = df.apply(f, axis=1)
df.tail()

C:\Users\JMadd\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR,Trend
Date,,,,,,,,,,,,,,,
2020-01-27,316.209991,321.260010,314.579987,316.600006,316.600006,7468100,328.293201,39.800736,39.610963,32.293929,-6.286475,-5.699115,-0.587360,-66.530010,-1
2020-01-28,318.049988,319.359985,314.000000,316.559998,316.559998,5502200,327.615601,39.775639,39.323804,37.478964,-6.322034,-5.823699,-0.498335,-64.521934,-1
2020-01-29,324.350006,327.179993,319.140015,322.019989,322.019989,12454700,327.376400,44.883662,39.474169,39.469645,-5.842293,-5.827418,-0.014875,-50.525550,-1
2020-01-30,320.500000,325.350006,317.640015,323.299988,323.299988,6022000,326.806400,46.039141,47.928693,42.242222,-5.297740,-5.721482,0.423742,-41.166436,-1
2020-01-31,321.750000,321.929993,316.989990,318.269989,318.269989,4996800,326.217200,42.287379,51.987834,46.463565,-5.211977,-5.619581,0.407604,-52.344512,-1


In [31]:
#I'm not sure if this is right, I would appreciate it if you double checked my work here

#5.3. Step 3: trading signal generation from trend analysis

Close = df.iloc[:,3]
df["UpTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5 + .5
df["DownTrend"] = (Close - Close.rolling('3 D').min())/(Close.rolling('3 D').max() - Close.rolling('3 D').min()) * .5

def g(row):
    if row['Trend'] == 1:
        val = row['UpTrend']
    elif row['Trend'] == -1:
        val = row['DownTrend']
    else:
        val = 0
    return val
df['Trade_Signal'] = df.apply(g, axis=1)
df.tail()

,Open,High,Low,Close,Adj Close,Volume,MA25,RSI,Stoch_K,Stoch_D,MACD,MACDSignal,MACDHist,WILLR,Trend,UpTrend,DownTrend,Trade_Signal
Date,,,,,,,,,,,,,,,,,,
2020-01-27,316.209991,321.260010,314.579987,316.600006,316.600006,7468100,328.293201,39.800736,39.610963,32.293929,-6.286475,-5.699115,-0.587360,-66.530010,-1,NaN,NaN,NaN
2020-01-28,318.049988,319.359985,314.000000,316.559998,316.559998,5502200,327.615601,39.775639,39.323804,37.478964,-6.322034,-5.823699,-0.498335,-64.521934,-1,0.5,0.0,0.0
2020-01-29,324.350006,327.179993,319.140015,322.019989,322.019989,12454700,327.376400,44.883662,39.474169,39.469645,-5.842293,-5.827418,-0.014875,-50.525550,-1,1.0,0.5,0.5
2020-01-30,320.500000,325.350006,317.640015,323.299988,323.299988,6022000,326.806400,46.039141,47.928693,42.242222,-5.297740,-5.721482,0.423742,-41.166436,-1,1.0,0.5,0.5
2020-01-31,321.750000,321.929993,316.989990,318.269989,318.269989,4996800,326.217200,42.287379,51.987834,46.463565,-5.211977,-5.619581,0.407604,-52.344512,-1,0.5,0.0,0.0


In [32]:
# df.count

In [33]:
#I added .tail(n=Length - 25) to omit values where the indicators = 0 since the machine learning agorithm trains on the values 
#of the indicator values and the MA15 indicator requires 25 previous close prices to calculate.
Length = (df.shape[0] - 25)


df2 = (df[['MA25', 'RSI', 'Stoch_K', 'Stoch_D', 'MACD', 'WILLR', 'Trade_Signal']].copy().tail(n=Length).copy())

df2 = df2.fillna(0)

df2.describe()

,MA25,RSI,Stoch_K,Stoch_D,MACD,WILLR,Trade_Signal
count,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000,1233.000000
mean,243.666060,54.252412,58.182146,58.178253,1.018105,-41.804268,0.367192
std,100.195049,12.604867,27.442183,26.632027,4.886728,29.434261,0.405876
min,117.517600,21.994722,3.036511,5.631216,-12.274485,-100.000000,0.000000
25%,143.736401,45.004095,34.064430,34.772775,-1.458254,-67.277909,0.000000
50%,230.151201,53.934569,63.262005,62.402013,0.862631,-36.607259,0.160715
75%,348.476800,62.643273,82.732234,82.016281,3.169401,-15.901727,0.671755
max,418.515598,92.963039,98.210606,96.755356,18.269471,-0.000000,1.000000


In [34]:
#5.5. Step 5: network structure creation and training using ELM

target_column = ['Trade_Signal'] 
predictors = list(set(list(df2.columns))-set(target_column))
#line 3 here fufills the data normalization of Step 5.4
df2[predictors] = (df2[predictors] - df2[predictors].min())/(df2[predictors].max() - df2[predictors].min())

#df2.describe()
#df2.info()
df2.head()
# df2

,MA25,RSI,Stoch_K,Stoch_D,MACD,WILLR,Trade_Signal
Date,,,,,,,
2015-03-11,0.113589,0.408098,0.279704,0.397274,0.401863,0.085919,0.000000
2015-03-12,0.114229,0.440207,0.166725,0.268254,0.401863,0.246129,0.847998
2015-03-13,0.114624,0.420464,0.171950,0.186813,0.401863,0.250002,0.000000
2015-03-16,0.115378,0.496756,0.332716,0.205269,0.401863,0.544944,0.000000
2015-03-17,0.116298,0.524579,0.478691,0.313880,0.401863,0.662921,1.000000


In [35]:
X = df2[predictors]

y = df2[target_column]


print(X.shape) ; print(y.shape)
# print(X)

(1233, 6)
(1233, 1)


In [36]:
Xindex = np.array([[1], [2], [3], [4], [5], [6]])

Xindex = Xindex.T

print(Xindex.shape); print(Xindex)

(1, 6)
[[1 2 3 4 5 6]]


In [37]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)

In [38]:
#  Functional Expansion Block

# cx = np.outer(X, Xindex).flatten(order='F')
# cx = np.sum(cx.reshape(-1, 6), axis=1)
# cx = np.tanh(cx)

#all on one line
cx = np.tanh(np.sum(np.outer(X, Xindex).flatten(order='F').reshape(-1, 6), axis=1))

print(cx.shape); print(cx)

(7398,)
[0.9336931  0.92710922 0.91307567 ... 1.         1.         1.        ]


In [39]:

cx = cx.reshape(1, len(cx) ).T
print(cx.shape); print(cx)

(7398, 1)
[[0.9336931 ]
 [0.92710922]
 [0.91307567]
 ...
 [1.        ]
 [1.        ]
 [1.        ]]


In [40]:
# cx = cx.reshape(-1, 6)
# print(cx.shape); print(cx)

In [41]:
# initialize random weights

synaptic_weights = np.random.random((1, (Length * 6)))

print(synaptic_weights.shape); print(synaptic_weights)

(1, 7398)
[[0.17216742 0.63996462 0.9954769  ... 0.5293378  0.98899777 0.82755193]]


In [42]:
#After expanding inputs the dimensions of of the cx matrix no longer matches the original output y so I can't get the difference between
# my weights and the error. Thoughts?

In [43]:
# z = np.multiply(cx, synaptic_weights)
z = np.dot(cx, synaptic_weights)
print(z); print(z.shape)

[[0.16075153 0.59753056 0.92946992 ... 0.49423906 0.9234204  0.77267953]
 [0.159618   0.5933171  0.92291582 ... 0.49075396 0.91690895 0.76723102]
 [0.15720188 0.58433613 0.90894574 ... 0.48332547 0.90302981 0.75561754]
 ...
 [0.17216742 0.63996462 0.9954769  ... 0.5293378  0.98899777 0.82755193]
 [0.17216742 0.63996462 0.9954769  ... 0.5293378  0.98899777 0.82755193]
 [0.17216742 0.63996462 0.9954769  ... 0.5293378  0.98899777 0.82755193]]
(7398, 7398)


In [20]:
#I'm working on figure 1 of the article and I have the expanded inputs CX, and the initialized weights as z
#I don't know how to read equation 2 or the Moore-Penros generalized inverse function of equation 5. If I'm reading this correctly
#(I'm not 100% sure I am), those functions are the key to using my CX expanded output values to create W

# I'm still new to ELM so if I'm missing something please let me know. 
#any help is appreciated
#Thank you so much !!!!

In [ ]:
#this is a for loop  I found online. It is not what is described in the article

for iteration in range(1000):
    z = np.dot(cx, synaptic_weights)
    sigmoid = 1/(1+np.exp(-z))
    error = (y - sigmoid)
print(error); print(error.shape)

In [ ]:
sigmoidDerivative = sigmoid * (1 - sigmoid)
print(sigmoidDerivative)

In [ ]:
synaptic_weights += np.dot(cx, error*sigmoidDerivative)
print(synaptic_weights)